# Прекод

# Сборный проект-4

Вам поручено разработать демонстрационную версию поиска изображений по запросу.

Для демонстрационной версии нужно обучить модель, которая получит векторное представление изображения, векторное представление текста, а на выходе выдаст число от 0 до 1 — покажет, насколько текст и картинка подходят друг другу.

### Описание данных

Данные доступны по [ссылке](https://code.s3.yandex.net/datasets/dsplus_integrated_project_4.zip).

В файле `train_dataset.csv` находится информация, необходимая для обучения: имя файла изображения, идентификатор описания и текст описания. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `train_images` содержатся изображения для тренировки модели.

В файле `CrowdAnnotations.tsv` — данные по соответствию изображения и описания, полученные с помощью краудсорсинга. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.
3. Доля людей, подтвердивших, что описание соответствует изображению.
4. Количество человек, подтвердивших, что описание соответствует изображению.
5. Количество человек, подтвердивших, что описание не соответствует изображению.

В файле `ExpertAnnotations.tsv` содержатся данные по соответствию изображения и описания, полученные в результате опроса экспертов. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.

3, 4, 5 — оценки трёх экспертов.

Эксперты ставят оценки по шкале от 1 до 4, где 1 — изображение и запрос совершенно не соответствуют друг другу, 2 — запрос содержит элементы описания изображения, но в целом запрос тексту не соответствует, 3 — запрос и текст соответствуют с точностью до некоторых деталей, 4 — запрос и текст соответствуют полностью.

В файле `test_queries.csv` находится информация, необходимая для тестирования: идентификатор запроса, текст запроса и релевантное изображение. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `test_images` содержатся изображения для тестирования модели.

## 1. Исследовательский анализ данных

Наш датасет содержит экспертные и краудсорсинговые оценки соответствия текста и изображения.

В файле с экспертными мнениями для каждой пары изображение-текст имеются оценки от трёх специалистов. Для решения задачи вы должны эти оценки агрегировать — превратить в одну. Существует несколько способов агрегации оценок, самый простой — голосование большинства: за какую оценку проголосовала большая часть экспертов (в нашем случае 2 или 3), та оценка и ставится как итоговая. Поскольку число экспертов меньше числа классов, может случиться, что каждый эксперт поставит разные оценки, например: 1, 4, 2. В таком случае данную пару изображение-текст можно исключить из датасета.

Вы можете воспользоваться другим методом агрегации оценок или придумать свой.

В файле с краудсорсинговыми оценками информация расположена в таком порядке:

1. Доля исполнителей, подтвердивших, что текст **соответствует** картинке.
2. Количество исполнителей, подтвердивших, что текст **соответствует** картинке.
3. Количество исполнителей, подтвердивших, что текст **не соответствует** картинке.

После анализа экспертных и краудсорсинговых оценок выберите либо одну из них, либо объедините их в одну по какому-то критерию: например, оценка эксперта принимается с коэффициентом 0.6, а крауда — с коэффициентом 0.4.

Ваша модель должна возвращать на выходе вероятность соответствия изображения тексту, поэтому целевая переменная должна иметь значения от 0 до 1.


In [1]:
import numpy as np
import pandas as pd
import random as rd
import os
import math
from math import ceil
import statistics
from functools import lru_cache
from PIL import Image
import PIL
import PIL.Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GroupShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from transformers import AutoTokenizer, AutoModel
from tensorflow.keras.models import Model
import torch
import torch.nn as nn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from torchvision import models
from torchvision.models import ResNet50_Weights
import spacy
import re

/Users/gurbanovsmac/anaconda3/envs/tf310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Импортируем данные, которые ранее были скачаны, и посмотрим, что они из себя представляют

In [2]:
train_dataset = pd.read_csv('to_upload/train_dataset.csv')
CrowdAnnotations = pd.read_csv('to_upload/CrowdAnnotations.tsv', sep='\t', header=None)
ExpertAnnotations = pd.read_csv('to_upload/ExpertAnnotations.tsv', sep='\t', header=None)
test_queries = pd.read_csv('to_upload/test_queries.csv', sep='|', index_col=0)
test_images = pd.read_csv('to_upload/test_images.csv', sep='|')

In [3]:
def display_table_info(name, data):
    print(name)
    display(data.head(10))
    data.info()
    print('')
    print('')
    print('')

display_table_info('train_dataset', train_dataset)
display_table_info('CrowdAnnotations', CrowdAnnotations)
display_table_info('ExpertAnnotations', ExpertAnnotations)
display_table_info('test_queries', test_queries)
display_table_info('test_images', test_images)

train_dataset


,image,query_id,query_text
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
1,1262583859_653f1469a9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
2,2447284966_d6bbdb4b6e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
4,2621415349_ef1a7e73be.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
5,3030566410_393c36a6c5.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
6,3155451946_c0862c70cb.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
7,3222041930_f642f49d28.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
8,343218198_1ca90e0734.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
9,3718964174_cb2dc1615e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       5822 non-null   object
 1   query_id    5822 non-null   object
 2   query_text  5822 non-null   object
dtypes: object(3)
memory usage: 136.6+ KB



CrowdAnnotations


,0,1,2,3,4
0,1056338697_4f7d7ce270.jpg,1056338697_4f7d7ce270.jpg#2,1.000000,3,0
1,1056338697_4f7d7ce270.jpg,114051287_dd85625a04.jpg#2,0.000000,0,3
2,1056338697_4f7d7ce270.jpg,1427391496_ea512cbe7f.jpg#2,0.000000,0,3
3,1056338697_4f7d7ce270.jpg,2073964624_52da3a0fc4.jpg#2,0.000000,0,3
4,1056338697_4f7d7ce270.jpg,2083434441_a93bc6306b.jpg#2,0.000000,0,3
5,1056338697_4f7d7ce270.jpg,2204550058_2707d92338.jpg#2,0.000000,0,3
6,1056338697_4f7d7ce270.jpg,2224450291_4c133fabe8.jpg#2,0.000000,0,3
7,1056338697_4f7d7ce270.jpg,2248487950_c62d0c81a9.jpg#2,0.333333,1,2
8,1056338697_4f7d7ce270.jpg,2307118114_c258e3a47e.jpg#2,0.000000,0,3
9,1056338697_4f7d7ce270.jpg,2309860995_c2e2a0feeb.jpg#2,0.000000,0,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47830 entries, 0 to 47829
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       47830 non-null  object 
 1   1       47830 non-null  object 
 2   2       47830 non-null  float64
 3   3       47830 non-null  int64  
 4   4       47830 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ MB



ExpertAnnotations


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5822 non-null   object
 1   1       5822 non-null   object
 2   2       5822 non-null   int64 
 3   3       5822 non-null   int64 
 4   4       5822 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 227.5+ KB



test_queries


,query_id,query_text,image
0,1177994172_10d143cb8d.jpg#0,"Two blonde boys , one in a camouflage shirt an...",1177994172_10d143cb8d.jpg
1,1177994172_10d143cb8d.jpg#1,Two boys are squirting water guns at each other .,1177994172_10d143cb8d.jpg
2,1177994172_10d143cb8d.jpg#2,Two boys spraying each other with water,1177994172_10d143cb8d.jpg
3,1177994172_10d143cb8d.jpg#3,Two children wearing jeans squirt water at eac...,1177994172_10d143cb8d.jpg
4,1177994172_10d143cb8d.jpg#4,Two young boys are squirting water at each oth...,1177994172_10d143cb8d.jpg
5,1232148178_4f45cc3284.jpg#0,A baby girl playing at a park .,1232148178_4f45cc3284.jpg
6,1232148178_4f45cc3284.jpg#1,A closeup of a child on a playground with adul...,1232148178_4f45cc3284.jpg
7,1232148178_4f45cc3284.jpg#2,A young boy poses for a picture in front of a ...,1232148178_4f45cc3284.jpg
8,1232148178_4f45cc3284.jpg#3,A young girl is smiling in front of the camera...,1232148178_4f45cc3284.jpg
9,1232148178_4f45cc3284.jpg#4,There is a little blond hair girl with a green...,1232148178_4f45cc3284.jpg


<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   query_id    500 non-null    object
 1   query_text  500 non-null    object
 2   image       500 non-null    object
dtypes: object(3)
memory usage: 15.6+ KB



test_images


,image
0,3356748019_2251399314.jpg
1,2887171449_f54a2b9f39.jpg
2,3089107423_81a24eaf18.jpg
3,1429546659_44cb09cbe2.jpg
4,1177994172_10d143cb8d.jpg
5,424307754_1e2f44d265.jpg
6,3044359043_627488ddf0.jpg
7,3396275223_ee080df8b5.jpg
8,2977379863_2e8d7a104e.jpg
9,634891010_9fa189effb.jpg


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   100 non-null    object
dtypes: object(1)
memory usage: 928.0+ bytes





Далее требуется добавить обобщённую экспертную оценку. В качестве итогового значения, согласно предложенному подходу, используется мнение большинства экспертов. В случае если оценки всех экспертов различаются, итоговое значение не присваивается, а соответствующие строки впоследствии исключаются из набора данных.

Для реализации этого шага будет разработана функция, принимающая на вход одну строку датасета и возвращающая результат обобщения экспертных оценок.

In [4]:
def result_exp(str_in):
    res=[0,0,0,0]
    for i in range(0,3):
        res[int(str_in[i+2]-1)]+=1
    if max(res) == 1:
        return None
    else:
        try:
            return res.index(max(res))+1
        except:
            print('ERROR')
            return None

Заполним столбец с результатом и удалим строки, в которых отсутствует результат

In [5]:
ExpertAnnotations[5] = ExpertAnnotations.apply(result_exp, axis=1)
ExpertAnnotations = ExpertAnnotations.dropna()

In [6]:
ExpertAnnotations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5696 entries, 0 to 5821
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5696 non-null   object 
 1   1       5696 non-null   object 
 2   2       5696 non-null   int64  
 3   3       5696 non-null   int64  
 4   4       5696 non-null   int64  
 5   5       5696 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 311.5+ KB


In [7]:
train_dataset = train_dataset.iloc[ExpertAnnotations.index]

## 2. Проверка данных

В некоторых странах, где работает ваша компания, действуют ограничения по обработке изображений: поисковым сервисам и сервисам, предоставляющим возможность поиска, запрещено без разрешения родителей или законных представителей предоставлять любую информацию, в том числе, но не исключительно тексты, изображения, видео и аудио, содержащие описание, изображение или запись голоса детей. Ребёнком считается любой человек, не достигший 16 лет.

В вашем сервисе строго следуют законам стран, в которых работают. Поэтому при попытке посмотреть изображения, запрещённые законодательством, вместо картинок показывается дисклеймер:

> This image is unavailable in your country in compliance with local laws
>

Однако у вас в PoC нет возможности воспользоваться данным функционалом. Поэтому все изображения, которые нарушают данный закон, нужно удалить из обучающей выборки.

Для выполнения данного требования сначала сформируем словарь, содержащий слова, относящиеся к детям и подросткам.

In [8]:
ban_dict = ['child', 'baby', 'boy', 'girl', 'teenager', 'schoolboy', 'youth', 'newborn']

Далее выполним преобразование текстовых описаний с использованием сформированного словаря.

In [9]:
nlp = spacy.load("en_core_web_sm")

def lemma_clear(text):
    lemm = nlp(text)
    lemm = " ".join([token.lemma_ for token in lemm])


    return " ".join(lemm.split())

In [10]:
corpus = train_dataset['query_text'].apply(lemma_clear)

Реализуем функцию, которая возвращает значение False, если в строке обнаружено запрещённое слово, и True — если такие слова отсутствуют.

In [11]:
def is_ban_word(text):
    for s in ban_dict: 
        if text.find(s) > -1:
            return False
    return True

In [12]:
train_dataset['lem_query_text'] = corpus
train_dataset['is_in_law'] = corpus.apply(is_ban_word)

Далее отберём все изображения, в описаниях которых встречаются «запрещённые» слова.

In [13]:
def spl(text):
    return text[:text.find('#')]

In [14]:
forb_images = list(train_dataset[train_dataset['is_in_law']==False]['query_id'].apply(spl).unique())

In [15]:
len(forb_images)

277

В результате проверки 277 изображений не прошли отбор, их список сохранён в переменной forb_images. Далее удалим из обучающей выборки все «запретные» описания, а также изображения, в описаниях которых встречаются запрещённые слова.

In [16]:
def is_not_forb(text):
    if text in forb_images:
        return False
    else:
        return True
    
train_dataset_clear = train_dataset[train_dataset['is_in_law']]
train_dataset_clear = train_dataset_clear[train_dataset_clear['image'].apply(is_not_forb)]

Таким образом, из обучающей выборки были исключены все данные, связанные с детьми.

Далее добавим к изображениям экспертные оценки. Ранее была получена обобщённая экспертная оценка. Поскольку экспертные оценки представлены в четырёх уровнях (от «полностью не соответствует» до «полностью соответствует»), тогда как оценки, полученные с аутсорса, принимают значения в диапазоне от 0 до 1 (доля респондентов, посчитавших описание соответствующим изображению), требуется привести экспертные оценки к сопоставимому масштабу.

В данном случае применим следующее преобразование: из значения экспертной оценки вычитается 1, после чего результат делится на 3. В результате получаем следующие значения:

 - полностью не соответствует — 0;

 - скорее не соответствует — 0.(3);

 - скорее соответствует — 0.(6);

 - полностью соответствует — 1.

Это преобразование позволяет объединить экспертные оценки с оценками аутсорса, используя коэффициенты значимости 0.6 и 0.4 соответственно.

In [17]:
display(ExpertAnnotations.head())
display(CrowdAnnotations.head())
#train_dataset_clear.head()

ExpertAnnotations[6] = (ExpertAnnotations[5]-1)/3
train_data = ExpertAnnotations.merge(CrowdAnnotations, left_on=[0, 1], right_on=[0, 1])[[0, 1, 6, '2_y']]
train_data['res'] = train_data[6]*0.6 + train_data['2_y']*0.4
train_data

,0,1,2,3,4,5
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,1,1,1,1.0
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,1,1,2,1.0
2,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,1,1,2,1.0
3,1056338697_4f7d7ce270.jpg,3207358897_bfa61fa3c6.jpg#2,1,2,2,2.0
4,1056338697_4f7d7ce270.jpg,3286822339_5535af6b93.jpg#2,1,1,2,1.0


,0,1,2,3,4
0,1056338697_4f7d7ce270.jpg,1056338697_4f7d7ce270.jpg#2,1.0,3,0
1,1056338697_4f7d7ce270.jpg,114051287_dd85625a04.jpg#2,0.0,0,3
2,1056338697_4f7d7ce270.jpg,1427391496_ea512cbe7f.jpg#2,0.0,0,3
3,1056338697_4f7d7ce270.jpg,2073964624_52da3a0fc4.jpg#2,0.0,0,3
4,1056338697_4f7d7ce270.jpg,2083434441_a93bc6306b.jpg#2,0.0,0,3


,0,1,6,2_y,res
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,0.000000,0.000000,0.000000
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,0.000000,0.000000,0.000000
2,1056338697_4f7d7ce270.jpg,434792818_56375e203f.jpg#2,0.000000,0.000000,0.000000
3,1084040636_97d9633581.jpg,256085101_2c2617c5d0.jpg#2,0.666667,0.333333,0.533333
4,1084040636_97d9633581.jpg,3396157719_6807d52a81.jpg#2,0.333333,0.000000,0.200000
...,...,...,...,...,...
2253,979383193_0a542a059d.jpg,3244747165_17028936e0.jpg#2,0.333333,0.000000,0.200000
2254,979383193_0a542a059d.jpg,3482062809_3b694322c4.jpg#2,0.333333,0.000000,0.200000
2255,997722733_0cb5439472.jpg,2985679744_75a7102aab.jpg#2,0.000000,0.000000,0.000000
2256,997722733_0cb5439472.jpg,3150742439_b8a352e1e0.jpg#2,0.000000,0.000000,0.000000


In [18]:
train_dataset_clear = train_dataset_clear.merge(train_data, left_on=['image', 'query_id'], right_on=[0, 1])[train_dataset_clear.columns]
train_data = train_dataset_clear.merge(train_data, left_on=['image', 'query_id'], right_on=[0, 1])[train_data.columns]

## 3. Векторизация изображений

Перейдём к векторизации изображений.

Самый примитивный способ — прочесть изображение и превратить полученную матрицу в вектор. Такой способ нам не подходит: длина векторов может быть сильно разной, так как размеры изображений разные. Поэтому стоит обратиться к свёрточным сетям: они позволяют "выделить" главные компоненты изображений. Как это сделать? Нужно выбрать какую-либо архитектуру, например ResNet-18, посмотреть на слои и исключить полносвязные слои, которые отвечают за конечное предсказание. При этом можно загрузить модель данной архитектуры, предварительно натренированную на датасете ImageNet.

In [19]:
resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)

for param in resnet.parameters():
    param.requires_grad_(False)
    
print(list(resnet.children())) 

modules = list(resnet.children())[:-1]
resnet = nn.Sequential(*modules) 

resnet.eval() 

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False), Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=Tr

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [20]:
from torchvision import transforms
norm = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

preprocess = transforms.Compose([
    transforms.Resize(150),
    transforms.CenterCrop(140),
    transforms.ToTensor(),
    norm,
]) 

In [21]:
img = Image.open("to_upload/train_images/1056338697_4f7d7ce270.jpg").convert('RGB') 

In [22]:
image_tensor = preprocess(img)
output_tensor = resnet(image_tensor.unsqueeze(0)).flatten()
output_tensor.size()
#output_tensor

torch.Size([2048])

Размер выходного вектора без применения пуллинга оказался слишком большим, поэтому он был уменьшен с помощью операции пуллинга для сокращения затрат памяти.

Далее преобразуем все изображения в датасете в векторные представления с использованием данной модели. Для этого реализуем соответствующую функцию.

In [23]:
def vect_gen_test(string):
    img = Image.open(path+string[0]).convert('RGB')
    image_tensor = preprocess(img)
    output_tensor = resnet(image_tensor.unsqueeze(0)).flatten()
    output_tensor.size()
    return output_tensor.numpy()

def vect_gen_train(string):
    img = Image.open(path+string['image']).convert('RGB')
    image_tensor = preprocess(img)
    output_tensor = resnet(image_tensor.unsqueeze(0)).flatten()
    output_tensor.size()
    return output_tensor.numpy()

In [24]:
path="to_upload/train_images/"
train_img_vec_arr = np.array(train_data.apply(vect_gen_test, axis=1))

## 4. Векторизация текстов

Следующий этап — векторизация текстов. Вы можете поэкспериментировать с несколькими способами векторизации текстов:

- tf-idf
- word2vec
- \*трансформеры (например Bert)

\* — если вы изучали трансформеры в спринте Машинное обучение для текстов.


Использоваем tf-idf

In [25]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gurbanovsmac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
corpus = train_dataset_clear['lem_query_text']

count_tf_idf = TfidfVectorizer(stop_words=list(stopwords)) 

tf_idf_train = count_tf_idf.fit_transform(corpus) 

print("Размер матрицы:", tf_idf_train.shape)

Размер матрицы: (1467, 757)


Теперь можно векторизируем тексты test_queries

In [27]:
corpus = test_queries['query_text'].apply(lemma_clear)
test_queries['lem_query_text'] = corpus
corpus = test_queries['lem_query_text']

In [28]:
tf_idf_test = count_tf_idf.transform(corpus) 

In [29]:
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (500, 757)


## 5. Объединение векторов

Подготовьте данные для обучения: объедините векторы изображений и векторы текстов с целевой переменной.

In [30]:
train_mass = np.hstack([np.vstack(train_img_vec_arr), tf_idf_train.toarray()])

## 6. Обучение модели предсказания соответствия

Для обучения разделите датасет на тренировочную и тестовую выборки. Простое случайное разбиение не подходит: нужно исключить попадание изображения и в обучающую, и в тестовую выборки.
Для того чтобы учесть изображения при разбиении, можно воспользоваться классом [GroupShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html) из библиотеки sklearn.model_selection.

Код ниже разбивает датасет на тренировочную и тестовую выборки в пропорции 7:3 так, что строки с одинаковым значением 'group_column' будут содержаться либо в тестовом, либо в тренировочном датасете.

```
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
train_indices, test_indices = next(gss.split(X=df.drop(columns=['target']), y=df['target'], groups=df['group_column']))
train_df, test_df = df.loc[train_indices], df.loc[test_indices]

```

Какую модель использовать — выберите самостоятельно. Также вам предстоит выбрать метрику качества либо реализовать свою.

In [31]:
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
train_indexes, test_indexes = next(gss.split(X=train_mass, y=train_data['res'], groups=train_dataset_clear['image']))
#train_df, test_df = df.loc[train_indices], df.loc[test_indices]

X_train = train_mass[train_indexes]
X_test = train_mass[test_indexes]
y_train = train_data['res'].loc[train_indexes]
y_test = train_data['res'].loc[test_indexes]

Так создадим и обучим линейную регрессию.

In [32]:
model_lr = LinearRegression(positive=True)
model_lr.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,True


In [33]:
scaler = MinMaxScaler()

y_pred = scaler.fit_transform(
    model_lr.predict(X_test).reshape(-1, 1)
)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("RMSE кросс-валидации:", rmse)

RMSE кросс-валидации: 0.391705364386589


Отлично, исходная точка определена. Теперь перейдём к построению полносвязной нейронной сети.

In [34]:
X_train = torch.FloatTensor(np.array(X_train))
X_test = torch.FloatTensor(np.array(X_test))
y_train = torch.FloatTensor(np.array(y_train))
y_test = torch.FloatTensor(np.array(y_test))

In [35]:
X_train.shape

torch.Size([1020, 2805])

In [36]:
class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, n_hidden_neurons_4, n_out_neurons):
            super(Net, self).__init__()
            
            self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
            self.act1 = nn.ReLU()
            #self.do1 = nn.Dropout(p=0.7)
            self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
            self.act2 = nn.ReLU()
            self.fc3 = nn.Linear(n_hidden_neurons_2, n_hidden_neurons_3)
            self.act3 = nn.ReLU()
            self.fc4 = nn.Linear(n_hidden_neurons_3, n_hidden_neurons_4)
            self.act4 = nn.ReLU()
            self.fc5 = nn.Linear(n_hidden_neurons_4, n_out_neurons)
            self.act5 = nn.Sigmoid()
            
            nn.init.kaiming_uniform_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc2.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc3.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc4.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc5.weight, mode='fan_in', nonlinearity='sigmoid')
            
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        #x = self.do1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        x = self.act3(x)
        x = self.fc4(x)
        x = self.act4(x)
        x = self.fc5(x)
        x = self.act5(x)
        
        return x

In [37]:
n_in_neurons = 2805
n_hidden_neurons_1 = 1200
n_hidden_neurons_2 = 600
n_hidden_neurons_3 = 200
n_hidden_neurons_4 = 10
n_out_neurons = 1



net = Net(n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_hidden_neurons_3, n_hidden_neurons_4, n_out_neurons)

#net = nn.Sequential(
#    nn.Linear(n_in_neurons, n_hidden_neurons_1),
#    nn.ReLU(),
#    nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
#    nn.ReLU(),
#    nn.Linear(n_hidden_neurons_2, n_out_neurons)
#)

optimizer2 = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.8)

optimizers = [optimizer2]

loss = nn.MSELoss()

In [38]:
num_epochs = 900
batch_size = 150
num_batches = ceil(len(X_train)/batch_size)

for optimizer in optimizers:
    optimizer_rmse = []
    print(f'Оптимизатор {optimizer}:')

    for epoch in range(num_epochs):
        net.train()
        order = np.random.permutation(len(X_train))
        
        for batch_idx in range(num_batches):
            
            start_index = batch_idx * batch_size
            optimizer.zero_grad()

            batch_indexes = order[start_index : start_index + batch_size]
            X_batch = X_train[batch_indexes]
            y_batch = y_train[batch_indexes]

            preds = net.forward(X_batch.float()).flatten()

            loss_value = loss(preds, y_batch)

            loss_value.backward()

            optimizer.step()

        if epoch % 10 == 0 or epoch == num_epochs - 1:
            with torch.no_grad():
                net.eval()
                test_preds = net.forward(X_test).flatten()
                #print(test_preds.numpy())
                ans = round(float(torch.sqrt(loss(test_preds, y_test))), 2)
                print(f'Метрика RMSE на {epoch} итерации =', ans)
                optimizer_rmse.append(ans)
    print()
    print(f'Лучшая метрика RMSE = {min(optimizer_rmse)} обнаружена на {optimizer_rmse.index(min(optimizer_rmse))} эпохе')
    print(f'Средняя метрика RMSE = {round(np.mean(optimizer_rmse), 2)}')
    print('---------------------------------------------------------------------------------------------', '\n')

Оптимизатор SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    momentum: 0.8
    nesterov: False
    weight_decay: 0
):
Метрика RMSE на 0 итерации = 0.35
Метрика RMSE на 10 итерации = 0.34
Метрика RMSE на 20 итерации = 0.34
Метрика RMSE на 30 итерации = 0.34
Метрика RMSE на 40 итерации = 0.34
Метрика RMSE на 50 итерации = 0.33
Метрика RMSE на 60 итерации = 0.33
Метрика RMSE на 70 итерации = 0.33
Метрика RMSE на 80 итерации = 0.33
Метрика RMSE на 90 итерации = 0.33
Метрика RMSE на 100 итерации = 0.33
Метрика RMSE на 110 итерации = 0.33
Метрика RMSE на 120 итерации = 0.33
Метрика RMSE на 130 итерации = 0.33
Метрика RMSE на 140 итерации = 0.33
Метрика RMSE на 150 итерации = 0.33
Метрика RMSE на 160 итерации = 0.33
Метрика RMSE на 170 итерации = 0.33
Метрика RMSE на 180 итерации = 0.33
Метрика RMSE на 190 итерации = 0.33
Метрика RMSE на 200 итерации = 0.33
Метрика RMSE на 210 итерации = 0.33
Метрика RMS

## 7. Тестирование модели

Настало время протестировать модель. Для этого получите эмбеддинги для всех тестовых изображений из папки `test_images`, выберите случайные 10 запросов из файла `test_queries.csv` и для каждого запроса выведите наиболее релевантное изображение. Сравните визуально качество поиска.

Для начала сформируем массив эмбеддингов. Для этого воспользуемся ранее реализованной функцией vect_gen_train.

In [39]:
path="to_upload/test_images/"
test_images_vec = np.vstack(np.array(test_images.apply(vect_gen_train, axis=1)))

Реализуем функцию, возвращающую список из десяти случайно выбранных описаний.

In [40]:
def ten_texts():
    texts = []
    for i in range(0,10):
        texts.append(test_queries.iloc[rd.randint(0, 499)]['query_text'])
    return texts

Теперь напишем функцию, которая подготовит текстовое описание ко входу в модель. Для этого тестовое описание векторизуется и соединяется с каждым из подготовленых изображений. За одно проверим тут текстовый запрос на наличие "запретных" слов

In [41]:
def conv_text(text):
    corpus_one = lemma_clear(text)
    corpus = []
    #print(corpus_one)
    #print(is_ban_word(corpus_one))
    if not is_ban_word(corpus_one):
        return None
    else:
        #tf_idf_one = count_tf_idf.transform(corpus)
        for i in range(0,100):
            corpus.append(corpus_one)
        tf_idf_one = count_tf_idf.transform(corpus)
        return(np.hstack([np.vstack(test_images_vec), tf_idf_one.toarray()]))

Теперь можно перейти к этапу тестирования. Для каждого из 10 отобранных описаний будет выведено само описание и изображение, набравшее наибольшее количество «очков».

In [43]:
for item in ten_texts():
    print(item)
    try:
        mass = conv_text(item)
        res = net.forward(torch.FloatTensor(mass)).flatten().detach().numpy()
        print(res.argmax())
    except:
        print('Наличие запретного слова в запросе!')

A white dog and a black dog in a field .
27
A man who is dressed like a wrestler walks while people take his picture .
27
Two people , one in blue and one in red , stand in front of a building .
27
A young boy is holding a PlayStation controller over his head .
Наличие запретного слова в запросе!
There is an older white haired lady and a bulldog sitting on a tree stump .
27
a red covered boat racing across the water
27
A little girl with her hands in the air is sitting on a man 's back while he is laying on a blanket .
Наличие запретного слова в запросе!
A watercraft speeds through the water .
27
A cyclist riding on their front wheel on the asphalt .
27
Three people are looking into photographic equipment .
27


Поскольку модель во всех случаях выдаёт одно и то же изображение, проведение визуальной оценки качества предсказаний не представляется целесообразным. Решить данную проблему самостоятельно не удалось. В связи с этим по рекомендации преподавателя работа была направлена на ревью с целью получения обратной связи и рекомендаций по возможным способам устранения данной проблемы.

## 8. Выводы

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Исследовательский анализ данных выполнен
- [ ]  Проверены экспертные оценки и краудсорсинговые оценки
- [ ]  Из датасета исключены те объекты, которые выходят за рамки юридических ограничений
- [ ]  Изображения векторизованы
- [ ]  Текстовые запросы векторизованы
- [ ]  Данные корректно разбиты на тренировочную и тестовую выборки
- [ ]  Предложена метрика качества работы модели
- [ ]  Предложена модель схожести изображений и текстового запроса
- [ ]  Модель обучена
- [ ]  По итогам обучения модели сделаны выводы
- [ ]  Проведено тестирование работы модели
- [ ]  По итогам тестирования визуально сравнили качество поиска